In [91]:
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, display_html, clear_output
import ipywidgets as widgets

In [92]:
try:
    model_9 = joblib.load('C:/Users/jim20/Desktop/Thesis Final/models_full/xgboost_model_9.pkl')
    model_22 = joblib.load('C:/Users/jim20/Desktop/Thesis Final/models_full/xgboost_model_22.pkl')
    print("Models loaded successfully")
except FileNotFoundError as e:
    print(f'Model file not found: {e}')
except Exception as e:
    print(f'Error loading models: {e}')


Models loaded successfully


In [93]:
dataset = pd.read_csv('C:/Users/jim20/Desktop/Thesis Final/data_full/meaningful.csv')
dataset.drop(columns=['DeathStatus_new', 'YearsToDeath', 'Hospital'], inplace=True)

REQUIRED_FEATURES = [
    'Agerand', 'DischargeCategoryHF', 'Durationofhospitalizationdays',
    'HFnew', 'DM', 'LVEFrand', 'SignVHD_012', 'BUNdis', 'HBdis'
]

EXPECTED_FEATURES = dataset.columns.tolist()

print(EXPECTED_FEATURES)
print(REQUIRED_FEATURES)

['Agerand', 'DischargeCategoryHF', 'HFnew', 'AF', 'LungDisease', 'DM', 'Sex', 'Stroke', 'Durationofhospitalizationdays', 'SignVHD_012', 'LVEFrand', 'LBBB_adm', 'HBdis', 'K_relchange_abs', 'NA_relchange_abs', 'TropDiff', 'GFR_absolute_relative_change_percent', 'BUNelev', 'BUNadm', 'BUNdis', 'TropChange_rel', 'NP_combined']
['Agerand', 'DischargeCategoryHF', 'Durationofhospitalizationdays', 'HFnew', 'DM', 'LVEFrand', 'SignVHD_012', 'BUNdis', 'HBdis']


In [94]:
CAT_FEATURES = ['HFnew', 'DM', 'SignVHD_012', 'Sex', 'LungDisease', 'Stroke', 'AF', 'LBBB_adm']

In [95]:
header = widgets.HTML(
    value="<h1 style='text-align:center; color:#2E86C1;'>Patient Survival Prediction</h1>"
          "<p style='text-align:center; color:#5D6D7E;'>Please enter the patient's details below.</p><hr>"
)

# --- 2. Define Input Widgets for Each Feature ---
input_widgets = {}
common_style = {'description_width': '320px'}
common_layout = {'width': '95%', 'margin': '8px 0'}

# 1. Agerand
input_widgets['Agerand'] = widgets.IntText(
    description='Age (Years):',
    style=common_style, layout=common_layout, value=60
)

# 2. DischargeCategoryHF
input_widgets['DischargeCategoryHF'] = widgets.Dropdown(
    description='Primary Discharge Diagnosis:',
    options=['Heart Failure', 'Valvular Heart Disease', 'Coronary Artery Disease', 'Tachyarrhythmia',
             'Bradyarrhythmia', 'Pulmonary hypertension', 'Other'],
    style=common_style, layout=common_layout, value='Other'
)

#3. Durationofhospitalizationdays
input_widgets['Durationofhospitalizationdays'] = widgets.IntText(
    description='Duration of Hospitalization (Days):',
    style=common_style, layout=common_layout, value=1
)

# 4. HFnew
input_widgets['HFnew'] = widgets.Dropdown(
    description='Heart Failure (No=0, Yes=1):',
    options=[('No', 0), ('Yes', 1)],
    style=common_style, layout=common_layout, value=0
)

# 5. DM
input_widgets['DM'] = widgets.Dropdown(
    description='Diabetes Mellitus (0=No, 1=Yes):',
    options=[('No', 0), ('Yes', 1)],
    style=common_style, layout=common_layout, value=0
)

# 6. LVEFrand
input_widgets['LVEFrand'] = widgets.FloatText(
    description='LVEF (%):',
    min=0,
    style=common_style, layout=common_layout, value=50.0
)

# 7. SignVHD_012
input_widgets['SignVHD_012'] = widgets.Dropdown(
    description='Moderate or Severe Valvular Heart Disease:',
    options=[('None', 0), ('Single Valve', 1), ('Multiple Valves', 2)],
    style=common_style, layout=common_layout, value=0
)

# 8. Blood Urea at Discharge
input_widgets['BloodUrea_dis'] = widgets.FloatText(
    description='Blood Urea at Discharge (mg/dL):',
    style=common_style, layout=common_layout, value=38.5  # example
)

# 9. Hemoglobin at Discharge
input_widgets['Hemoglobin_dis'] = widgets.FloatText(
    description='Hemoglobin at Discharge (g/dL):',
    style=common_style, layout=common_layout, value=13.0  # example
)

# 10. Sex
input_widgets['Sex'] = widgets.Dropdown(
    description='Sex:',
    options=[('Male', 0), ('Female', 1)],
    style=common_style, layout=common_layout, value=0
)

# 11. LungDisease
input_widgets['LungDisease'] = widgets.Dropdown(
    description='Lung Disease:',
    options=[('No', 0), ('Yes', 1)],
    style=common_style, layout=common_layout, value=0
)


# 12. Stroke
input_widgets['Stroke'] = widgets.Dropdown(
    description='Prior Stroke:',
    options=[('No', 0), ('Yes', 1)],
    style=common_style, layout=common_layout, value=0
)

# 13. AF
input_widgets['AF'] = widgets.Dropdown(
    description='Atrial Fibrillation:',
    options=[('No', 0), ('Yes', 1)],
    style=common_style, layout=common_layout, value=0
)

# 14. LBBB_adm
input_widgets['LBBB_adm'] = widgets.Dropdown(
    description='Left Bundle Branch Block:',
    options=[('No', 0), ('Yes', 1)],
    style=common_style, layout=common_layout, value=0
)

# 15. BNP
input_widgets['BNP_close_adm'] = widgets.FloatText(
    description='BNP (pg/ml) closest to admission:',
    style=common_style, layout=common_layout, value=100.0  # example value
)

# 16. NTproBNP
input_widgets['NTproBNP_close_adm'] = widgets.FloatText(
    description='NTproBNP (pg/ml) closest to admission:',
    style=common_style, layout=common_layout, value=1000.0  # example value
)

# 17. Blood Urea at Admission
input_widgets['BloodUrea_adm'] = widgets.FloatText(
    description='Blood Urea at Admission (mg/dL):',
    style=common_style, layout=common_layout, value=40.0  # example value
)

# 18. Creatinine at Admission
input_widgets['Creatinine_adm'] = widgets.FloatText(
    description='Creatinine at Admission (mg/dL):',
    style=common_style, layout=common_layout, value=1.2  # example value
)

# 19. Creatinine at Discharge
input_widgets['Creatinine_dis'] = widgets.FloatText(
    description='Creatinine at Discharge (mg/dL):',
    style=common_style, layout=common_layout, value=1.0  # example value
)

# 20. Potassium at Admission
input_widgets['Potassium_adm'] = widgets.FloatText(
    description='Potasium at admission (mmol/l):',
    style=common_style, layout=common_layout, value=4.0  # example value
)

# 21. Potasium at discharge (mmol/l)
input_widgets['Potassium_dis'] = widgets.FloatText(
    description='Potasium at discharge (mmol/l):',
    style=common_style, layout=common_layout, value=4.2  # example value
)

# 22. Sodium at admission (mmol/l)
input_widgets['Sodium_adm'] = widgets.FloatText(
    description='Sodium at admission (mmol/l):',
    style=common_style, layout=common_layout, value=140.0  # example value
)

# 23. Sodium at discharge (mmol/l)
input_widgets['Sodium_dis'] = widgets.FloatText(
    description='Sodium at discharge (mmol/l):',
    style=common_style, layout=common_layout, value=138.0  # example value
)

# 24. Troponin I at admission (ng/ml)
input_widgets['TroponinI_adm'] = widgets.FloatText(
    description='Troponin I at admission (ng/ml):',
    style=common_style, layout=common_layout, value=0.05  # example value
)

# 25. Troponin T at admission (pg/ml or ng/L)
input_widgets['TroponinT_adm'] = widgets.FloatText(
    description='Troponin T at admission (pg/ml or ng/L):',
    style=common_style, layout=common_layout, value=0.1  # example value
)

# 26. Troponin I at discharge (ng/ml)
input_widgets['TroponinI_dis'] = widgets.FloatText(
    description='Troponin I at discharge (ng/ml):',
    style=common_style, layout=common_layout, value=0.03  # example value
)

# 27. Troponin T at discharge (pg/ml or ng/L)
input_widgets['TroponinT_dis'] = widgets.FloatText(
    description='Troponin T at discharge (pg/ml or ng/L):',
    style=common_style, layout=common_layout, value=0.05  # example value
)

# 28. Peak Troponin I during hospitalization (ng/ml)
input_widgets['TroponinI_peak'] = widgets.FloatText(
    description='Peak Troponin I during hospitalization (ng/ml):',
    style=common_style, layout=common_layout, value=0.1  # example value
)

# 29.Peak Troponin T during hospitalization (ng/ml)
input_widgets['TroponinT_peak'] = widgets.FloatText(
    description='Peak Troponin T during hospitalization (pg/ml or ng/L):',
    style=common_style, layout=common_layout, value=0.15  # example value
)

# --- 3. Arrange them in a VBox ---
visible_input_keys = [
    'Agerand', 'DischargeCategoryHF', 'Durationofhospitalizationdays', 'HFnew',
    'DM', 'LVEFrand', 'DischargeCategoryHF', 'BloodUrea_dis', 'Hemoglobin_dis',
    'Sex', 'LungDisease', 'Stroke', 'AF', 'LBBB_adm', 'BNP_close_adm',
    'NTproBNP_close_adm', 'BloodUrea_adm', 'Creatinine_adm', 'Creatinine_dis',
    'Potassium_adm', 'Potassium_dis', 'Sodium_adm', 'Sodium_dis',
    'TroponinI_adm', 'TroponinT_adm', 'TroponinI_dis', 'TroponinT_dis',
    'TroponinI_peak', 'TroponinT_peak'
]

form_items = [input_widgets[key] for key in visible_input_keys if key in input_widgets]

input_form_vbox = widgets.VBox(form_items)
input_form_vbox.layout.padding = '20px'
input_form_vbox.layout.border = '1px solid #AED6F1' # Light blue border
input_form_vbox.layout.border_radius = '8px'      # Rounded corners
input_form_vbox.layout.background_color = '#F9E79F' # Light yellow background for the form
# Set a width for the form itself to enable centering it
input_form_vbox.layout.width = '60%' # Or a fixed width like '700px'
input_form_vbox.layout.min_width = '500px' # Prevent it from becoming too narrow

In [96]:
predict_button = widgets.Button(
    description="Get Survival Prediction",
    button_style='primary', # 'primary' is often blue
    icon='calculator',      # FontAwesome icon
    layout={'width': 'auto', 'margin': '25px 0 0 0'} # Margin top, auto horizontal margins for centering if VBox align_items is center
)
# predict_button.style.button_color = '#58D68D' # Custom green color for button

# --- 5. Output Area ---
output_area = widgets.Output()
output_area.layout.margin = '20px 0 0 0' # Margin top for spacing from the button
output_area.layout.width = '90%' # Make output area responsive

# --- 6. Main Container to Hold and Center Everything ---
# This VBox will hold the header, the input_form_vbox, the button, and the output area.
# We then center this overall_container.
overall_container = widgets.VBox(
    [header, input_form_vbox, predict_button, output_area],
    layout=widgets.Layout(
        align_items='center', # Center children (like the button and the form VBox) horizontally
        width='90%',          # Overall container takes 90% of available width
        margin='0 auto',      # CSS trick to center the overall_container itself
        padding='10px'
    )
)

In [97]:
def on_predict_button_clicked(b):
    with output_area:
        clear_output(wait=True) # Clear previous results
        print("Collecting data and making prediction...")

        # A. Collect data from widgets
        patient_data_dict = {}
        all_valid = True
        for feature_name in EXPECTED_FEATURES:
            if feature_name in input_widgets:
                patient_data_dict[feature_name] = input_widgets[feature_name].value
            else:
                display_html(f"<p style='color:red;'>Error: Input for {feature_name} not found!</p>")
                all_valid = False
        
        if not all_valid:
            return

        # B. Convert to Pandas DataFrame
        import pandas as pd # Make sure pandas is imported
        patient_df_raw = pd.DataFrame([patient_data_dict])
        patient_df_for_model = patient_df_raw[EXPECTED_FEATURES] # Ensure correct order

        try:
            sex_mapping = {'male': 0, 'female': 1}
            discharge_mapping = {'Other': 0, 'ADHF_VHD_PH': 1}
            patient_df_for_model['Sex'] = patient_df_for_model['Sex'].map(sex_mapping)
            patient_df_for_model['DischargeCategoryHF'] = patient_df_for_model['DischargeCategoryHF'].map(discharge_mapping)
        except Exception as e:
            display_html(f"<p style='color:red;'>Error during preprocessing: {e}</p>")
            return

        # E. Make Prediction
        try:
            survival_functions = model.predict_survival_function(patient_df_for_model)
            patient_survival_function = survival_functions[0] # For a single patient input
            plt.figure(figsize=(10, 6))
            plt.step(patient_survival_function.x, patient_survival_function.y, where="post", color="green")
            plt.title("Predicted Patient Survival Function")
            plt.xlabel("Years To Death")
            plt.ylabel("Survival Probability")
            plt.ylim(0, 1.05)
            plt.xlim(left=0)
            plt.grid(True, linestyle='--', alpha=0.7)
            plt.show() # This will display the plot in the output_area
            # Placeholder text until plotting is implemented:
            display_html(f"<h3 style='color:green;'>Prediction Placeholder</h3>"
                         f"<p>Data collected. Preprocessing, prediction, and plotting logic needs to be implemented.</p>"
                         # f"<p>Example: Survival at time X: {patient_survival_function.predict_proba(X_value)}</p>"
                         )
        
        except Exception as e:
            display_html(f"<p style='color:red;'>Error during prediction/plotting: {e}</p>")
            import traceback
            print(traceback.format_exc()) # Print full traceback for debugging

        # For now, just show what was collected
        display_html(f"<h3 style='color:green;'>Data Collection Complete (Implement Prediction Logic)</h3>")
        for key, val in patient_data_dict.items():
            display_html(f"<b>{key}:</b> {val}")


# --- Link button to the function ---
predict_button.on_click(on_predict_button_clicked)
clear_output(wait=True)
# --- Display the Entire Interface ---
display(overall_container)